In [63]:
from databases import BDS, PosicaoDm1, Bawm,Crm,BaseExtrato,off_Reliance
from funcoes_datas import FuncoesDatas
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date
import numpy as np
import math
from emailer import Email, EmailLer
from pretty_html_table import build_table
from funcoes import CalculosRiscoPort

In [64]:
#Chama as classes do databases e as classes do funções data
rel_off_p = off_Reliance('Primeny')
rel_off_c = off_Reliance('Cubo')
crm = Crm()
bawm = Bawm()
datas = FuncoesDatas()
extrato = BaseExtrato()
bds = BDS()
fdt = FuncoesDatas()
calc = CalculosRiscoPort(offshore = True)

In [65]:
#Procura os parametros para rodar as funções

#Puxando a ultima data do mes anterior
data = bds.banco.hoje()
data = datetime.datetime.date(data)
hoje = fdt.workday(data, n_dias=-4)

#Traz os de_paras de mandatos (mandato anterior para o atual)
mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_mandato')
mandatos = mandatos[['Anterior','Novo']]
#mandatos = dict(mandatos.values)
    
# #Traz os de_paras de classificação gps para asset Allocation
dicio_classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacoes=dicio_classificacoes[['Classificação GPS','De Para - Asset Allocation']]
dicio_classificacoes = dict(dicio_classificacoes.values)

#Novos mandatos
novos_mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Novos_mandatos')
novos_mandatos= novos_mandatos[['Portfolio','Classe','Tático']]

#Trazer os produtos GPS e segregar o que é imobiliario (caso precise segregar o que é Reits / ILiquidos)
produtos_gps = crm.produtos_off()
produtos_gps =produtos_gps[['name','new_idmysql','new_tickerid','new_isin','new_idbds','new_idsistemaorigem','new_indgroupaxysidname']].rename(columns={'name':'Ativo','new_idmysql':'IdMysqlProduto','new_tickerid':'Ticker','new_isin':'ISIN','new_idsistemaorigem':'CodigoProduto'}).reset_index()
produtos_gps['ISIN'] = produtos_gps['ISIN'].str.lower()

#Trazendo o De Para de ativos e separando os dataframes para unir com a carteira
de_para_ativos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Tb De Para Subclass')

# #Trazendo todos os produtos da base Reliance
# produtos_rel= rel_off_p.produtos_off().drop(columns=['subclasse'])
# produtos_rel['ISIN'] = produtos_rel['ISIN'].str.lower()

#Trazendo o dicionario da classificação dos ativos dado pela area de investimentos (produtos reliance)
dicio_classificacao_investimentos_rel = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacao_investimentos_rel=dicio_classificacao_investimentos_rel[['Classificação Reliance-Investimentos','Classificação GPS - v1']]
dicio_classificacao_investimentos_rel = dict(dicio_classificacao_investimentos_rel.values)

#Dicionario Reliance Classe + Subclasse + Categoria
dicio_classe_sub_categoria = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classe_sub_categoria=dicio_classe_sub_categoria[['Classe_Subclasse_Categoria-Sistema','Classificação GPS - v2']]
dicio_classe_sub_categoria = dict(dicio_classe_sub_categoria.values)

#Saber a qual classe pertence cada subclasse
dicio_classe_subclasse_gps = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classe_subclasse_gps=dicio_classe_subclasse_gps[['Subclasse GPS','Classe']]
dicio_classe_subclasse_gps = dict(dicio_classe_subclasse_gps.values)

#Benchmarks
benchs = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
benchs=benchs[['De Para - Asset Allocation','Index_Bawm']]

#Puxa a carteira e faz os tratamentos para a mesma ser lida pelo robo
alocacao = pd.read_excel('pesos_carteira_rel.xlsx')


In [66]:
#Pegando o primeiro dia de cada mes
data = fdt.hoje()
data = data - relativedelta(days=30)
data = date(data.year, data.month, 1) 
#Não usar

data=pd.to_datetime(data)
data = data - relativedelta(days=30)
data = date(data.year, data.month, 1) 

#Pode seguir daqui

data=pd.to_datetime(data)

# #Carteira do ultimo extrato
# alocacao = rel_off_c.carteira_por_peso(data)
# alocacao['NomeClasse']=alocacao['NomeClasse'].str.lower()


In [67]:
#Converter as classes unificadas para as classes de asset allocation
del dicio_classificacoes[np.nan]
dicio_classificacoes = dict((k.lower(), v.lower()) for k, v in dicio_classificacoes.items()) 
alocacao['Classe']=alocacao['Classe'].apply(lambda x:dicio_classificacoes[x])

#Fazendo o merge com todos os possíveis portfolios
novos_mandatos['Classe'] = novos_mandatos['Classe'].str.lower()
novos_mandatos = novos_mandatos[novos_mandatos['Classe']!='alternativos long short asia']
novos_mandatos = novos_mandatos[novos_mandatos['Portfolio'].str.contains('MS') == False]

#Puxar os clientes para o for
clientes = alocacao['Nome_Cliente'].drop_duplicates().to_list()
clientes = [x for x in clientes if x is not None]

#Puxar os portfolios para o for
portfolios = novos_mandatos['Portfolio'].drop_duplicates().to_list()

#dicionario de classes - Deixando tudo minusculo e equiparando com as classes de asset allocation
dicio_classes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classes=dicio_classes[['Classe_Alocacao_sistema','Classe GPS']]
dicio_classes = dict(dicio_classes.values)

#Rentabilidade das carteiras
rentabilidade = rel_off_c.rentabilidade(data)
rentabilidade = rentabilidade.loc[rentabilidade.groupby('nome_reduzido').DataProcessamentoFim.idxmin()].reset_index(drop=True)
rentabilidade = rentabilidade[['nome_reduzido','PontosRisco','RetornoLiquido_NoAno','RetornoLiquido_12Meses']]


In [68]:
#Calcular o TE da carteira e dividir pela Volatilidade (Risco Ativo)

pos = pd.DataFrame()
tracking= []
#clientes = ['SHIBUYA JB']

for cliente in clientes:
    nome=cliente
    df = alocacao[alocacao['Nome_Cliente']==nome]
    df= pd.merge(left=df,right=novos_mandatos,on='Classe',how='right')
    df = df[df['Classe'].str.contains('jbfo') == False]
    df['Nome_Cliente']=df['Nome_Cliente'].fillna(nome)
    df['percentual_produto']= df['percentual_produto'].fillna(0)
    df = df.rename(columns={'percentual_produto':'Atual'})    
    df= df[['Portfolio','Classe','Tático','Atual']]
         
    for portfolio in portfolios:
        analise = df[df['Portfolio']==portfolio]
        analise = analise[['Classe','Tático','Atual']]
        analise['Classe']=analise['Classe'].str.lower()
        analise['dif']=abs(analise['Atual'] - analise['Tático'] )
        te=calc.tracking_error(analise)
        analise = analise.reset_index()
        port_model = analise[['Classe','Tático']].rename(columns={'Tático':'Atual'})
        port_atual = analise[['Classe','Atual']]
        vol_port = calc.vol_carteira(port_model)
        vol_atual = calc.vol_carteira(port_atual)
        te_volport = te/vol_port
        if te_volport >1:
            te_volport = 1
        else:
            te_volport
        mae = (analise['dif'].sum()).mean()/len(analise)  
        delta_vol = abs(vol_atual-vol_port)
        proxy = mae * te
        tracking.append({'Cliente': nome, 'portfolio': portfolio,'Proxy': proxy,'TE':te,'Delta_Vol':delta_vol})
        
        
    
df = pd.DataFrame.from_dict(tracking)    
df['Mandato/TE']=df.apply(lambda row : str(row['portfolio']) +' / '+ str(round(row['TE'],3)),axis=1)
df['Mandato/Proxy']=df.apply(lambda row : str(row['portfolio']) +' / '+ str(round(row['Proxy'],3)),axis=1)
df['Mandato/Delta_Vol']=df.apply(lambda row : str(row['portfolio']) +' / '+ str(round(row['Delta_Vol'],3)),axis=1)
df = df.sort_values(['Cliente', 'Proxy'], ascending=[True, True]).reset_index(drop=True)


port_recomendado = df.loc[df.groupby('Cliente').Proxy.idxmin()].reset_index(drop=True)
port_recomendado=pd.merge(left=rentabilidade,right=port_recomendado, left_on='nome_reduzido',right_on='Cliente',how='right')
port_recomendado = port_recomendado[['Cliente','RetornoLiquido_NoAno','PontosRisco','portfolio','TE','Proxy','Delta_Vol']].rename(columns={'RetornoLiquido_NoAno':'RetornoLiquido_2023','portfolio':'Mandato_Recomendado'})  
    

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

...

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vol'] = vols
C:\Temp\Foundation\funcoes.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CTR'] = df['Atual'] * df['Vol']
C:\Temp\Foundation\funcoes.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [62]:
rank=[]
df['verifica']=df['Cliente']==df['Cliente'].shift(1)


for idx,row in df.iterrows():
    if row['verifica']==False:
        n=1
    else:
        n=n+1
    rank.append(n)    
            
df['Ranking']=rank    
df = df.groupby(['Cliente','Ranking'])['Mandato/Proxy','Mandato/TE','Mandato/Delta_Vol'].aggregate('first').unstack().reset_index()
port_recomendado = pd.merge(left=port_recomendado,right=df,on='Cliente',how='left').sort_values(by=['Cliente']).dropna(subset=['Cliente'])

<ipython-input-62-dfc4470d9afb>:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(['Cliente','Ranking'])['Mandato/Proxy','Mandato/TE','Mandato/Delta_Vol'].aggregate('first').unstack().reset_index()
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [ ]:
df = df.groupby(['Cliente','Ranking'])['Mandato/Proxy','Mandato/TE','Mandato/Delta_Vol'].aggregate('first').unstack().reset_index()

In [69]:
port_recomendado.to_excel('port_rec.xlsx')

In [ ]:
pos = pd.DataFrame()
tracking= []
clientes = ['SHIBUYA JB']

for cliente in clientes:
    nome=cliente
    df = alocacao[alocacao['Nome_Cliente']==nome]
    df= pd.merge(left=df,right=novos_mandatos,on='Classe',how='right')
    df = df[df['Classe'].str.contains('jbfo') == False]
    df['Nome_Cliente']=df['Nome_Cliente'].fillna(nome)
    df['percentual_produto']= df['percentual_produto'].fillna(0)
    df = df.rename(columns={'percentual_produto':'Atual'})    
    df= df[['Portfolio','Classe','Tático','Atual']]
    
    for portfolio in portfolios:
        analise = df[df['Portfolio']==portfolio]
        analise = analise[['Classe','Tático','Atual']]
        analise['Classe']=analise['Classe'].str.lower()
        te=calc.tracking_error(analise)
        tracking.append({'Cliente': nome, 'portfolio': portfolio,'TE': te})
        
    
df = pd.DataFrame.from_dict(tracking)    
df['Mandato/TE']=df.apply(lambda row : str(row['portfolio']) +' / '+ str(round(row['TE'],3)),axis=1)
df = df.sort_values(['Cliente', 'TE'], ascending=[True, True]).reset_index(drop=True)


port_recomendado = df.loc[df.groupby('Cliente').TE.idxmin()].reset_index(drop=True)
port_recomendado=pd.merge(left=rentabilidade,right=port_recomendado, left_on='nome_reduzido',right_on='Cliente',how='right')
port_recomendado = port_recomendado[['Cliente','RetornoLiquido_NoAno','PontosRisco','portfolio','TE']].rename(columns={'RetornoLiquido_NoAno':'RetornoLiquido_2023','portfolio':'Mandato_Recomendado'})

In [ ]:
port_recomendado